In [1]:
import sim          # Library for coppelia communication       
import sympy as sp  # Library to work with symbolic variables

In [2]:
# Function to calculate symbolic denavit transformations from parameters

def symTfromDH(theta, d, a, alpha):

    Rz = sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, 0],
                   [sp.sin(theta), sp.cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = sp.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])
    ta = sp.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = sp.Matrix([[1, 0, 0, 0],
                   [0, sp.cos(alpha), -sp.sin(alpha), 0],
                   [0, sp.sin(alpha), sp.cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

## Calculate Denavit Transformations

In [81]:
# Robot dimensions
L1 = 300
L2 = 600
L3 = 400

q1 = sp.symbols('q1') # First rotational joint angle
q2 = sp.symbols('q2') # Second rotational joint angle
l1 = sp.symbols('l1') # First prismatic joint length

# Calculated denavit parameters
theta = [0,q1, q2 ,0 ]
d = [L1,0,0,l1]
a = [0,L2,L3,0]
alpha = [sp.pi/2,0,sp.pi/2,0]

# Partial transformations to end effector
T01 = symTfromDH(theta[0],d[0],a[0],alpha[0])
T12 = symTfromDH(theta[1],d[1],a[1],alpha[1])
T23 = symTfromDH(theta[2],d[2],a[2],alpha[2])
T34 = symTfromDH(theta[3],d[3],a[3],alpha[3])

# Final tranformation
T04 = T01*T12*T23*T34
T04.simplify()
T04

Matrix([
[cos(q1 + q2),  0,  sin(q1 + q2),        l1*sin(q1 + q2) + 600*cos(q1) + 400*cos(q1 + q2)],
[           0, -1,             0,                                                       0],
[sin(q1 + q2),  0, -cos(q1 + q2), -l1*cos(q1 + q2) + 600*sin(q1) + 400*sin(q1 + q2) + 300],
[           0,  0,             0,                                                       1]])

## Direct Kinematics

In [89]:
# Evaluate end effector transformation
#2.02122158  2.7754343   1.
values ={     
q1:0,         
q2:0,    
l1:0,           
}

T04.evalf(subs=values)

Matrix([
[1.0,    0,    0, 1000.0],
[  0, -1.0,    0,      0],
[  0,    0, -1.0,  300.0],
[  0,    0,    0,    1.0]])

In [7]:
T04

Matrix([
[cos(q1 + q2),  0,  sin(q1 + q2),        l1*sin(q1 + q2) + 600*cos(q1) + 400*cos(q1 + q2)],
[           0, -1,             0,                                                       0],
[sin(q1 + q2),  0, -cos(q1 + q2), -l1*cos(q1 + q2) + 600*sin(q1) + 400*sin(q1 + q2) + 300],
[           0,  0,             0,                                                       1]])

## Inverse Kinematics

In [85]:
# From dh matrix 

X = 31
Y = 0
Z = 33

def eqs(q):
    q1 = q[0]
    q2 = q[1]
    l1 = q[2]
    f = np.zeros(3)
    f[0] = l1*np.sin(q1 + q2) + 600*np.cos(q1) + 400*np.cos(q1 + q2) -X
    f[1] = 0 - Y
    f[2] = -l1*np.cos(q1 + q2) + 600*np.sin(q1) + 400*np.sin(q1 + q2) + 300 - Z
    return f

In [88]:
from scipy import optimize
import numpy as np

q=optimize.fsolve(eqs,[1,1,1])
print(q)
#print(f'q1 {q1} q2 {q2} l1 {l1} ')

[-2.02122158  2.7754343   1.        ]


In [21]:
# Solve position equations
print("calculando la cinemática inversa...")
q = sp.nsolve((eqx,eqy,eqz), (q1,q2,l1), (1,1,1), prec=3)
print (q)

calculando la cinemática inversa...


ZeroDivisionError: matrix is numerically singular

In [ ]:
# movemos el robot a la posición
print("moviendo a la posición objetivo...")
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_blocking)
time.sleep(1)